In [89]:
import pandas as pd
import json
import requests
import datetime as dt
from datetime import date, timedelta
import sys
from bs4 import BeautifulSoup
from datetime import datetime, timedelta


In [28]:
token = pd.read_csv('./access.csv').loc[0]['token']
start_date = date.today() - timedelta(days=3)
end_date = date.today() - timedelta(days=1)
start_date = start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

print(start_date, end_date)

2024-08-17 2024-08-19


아바티 로그인 및 데이터 수집
> iOS의 경우 1일 전 데이터가 안들어오는듯? 대시보드상에는 들어옴
`> 일단 API 요청했으니 구글시트로 temp

In [190]:
end_date = date.today().strftime('%Y-%m-%d')
end_date = date.today() - timedelta(days=8)
end_date = end_date.strftime('%Y-%m-%d')

In [188]:
print(end_date)

2024-08-21


In [192]:
data_url

'https://partner.avatye.com/api/cashButton/report/chart?publisherID=cd0209ad4e834551b8a4ebb0b25850f9&now=2024-08-13T00:00:00.000%2B09:00&appType=cashmore,cashmore_direct,cashmore_direct2'

In [191]:
#7일씩 가져옴. 마지막 날짜 기준
login_url = "https://partner.avatye.com/api/auth/check"
data_url = f"https://partner.avatye.com/api/cashButton/report/chart?publisherID=cd0209ad4e834551b8a4ebb0b25850f9&now={end_date}T00:00:00.000%2B09:00&appType=cashmore,cashmore_direct,cashmore_direct2"
login_data = {
    'emailID' : "iehong@hecto.co.kr",
    'password' : "cash!1234"
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0',
    'Content-Type': 'application/json', 
}
def get_login(login_url,json,headers):
    response = session.post(login_url, json=login_data, headers=headers)
    if response.status_code == 200:
        return response
    else:
        False
        
# 웹 페이지 URL
def get_data(data_url):
    url = data_url
    # 웹 페이지의 HTML 가져오기
    response = requests.get(url)
    return response.json()

def convert_timezone(utc) :
    utc_time = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%S.%fZ")
    kst_time = utc_time + timedelta(hours=9)
    return kst_time.strftime("%Y-%m-%d %H:%M:%S")

def get_avati_report(json_data):
    df = pd.json_normalize(
        json_data['chartData'],
        record_path=['data'],
        meta=['appID', 'appName'],
        errors='ignore'
    )

    df['actionDateTime'] = df['actionDateTime'].apply(convert_timezone)

    return df

with requests.Session() as session:
    response = get_login(login_url, json=login_data, headers=headers)
    if response == False:
        print("login_failed")
        sys.exit(1)
    else:
        print("login_success")
        pass
    
    json_data = get_data(data_url)

    result_data = get_avati_report(json_data)

result_data
    





login_success


,actionDateTime,dau,expectedRevenue,appID,appName
0,2024-08-06 00:00:00,14855,369035.0,6d5760ccfd9748d3a1ab2ffaa72c938f,발로소득
1,2024-08-07 00:00:00,14601,401671.0,6d5760ccfd9748d3a1ab2ffaa72c938f,발로소득
2,2024-08-08 00:00:00,14593,532851.0,6d5760ccfd9748d3a1ab2ffaa72c938f,발로소득
3,2024-08-09 00:00:00,14152,589654.0,6d5760ccfd9748d3a1ab2ffaa72c938f,발로소득
4,2024-08-10 00:00:00,14009,618737.0,6d5760ccfd9748d3a1ab2ffaa72c938f,발로소득
5,2024-08-11 00:00:00,14706,440983.0,6d5760ccfd9748d3a1ab2ffaa72c938f,발로소득
6,2024-08-12 00:00:00,15427,650655.0,6d5760ccfd9748d3a1ab2ffaa72c938f,발로소득
7,2024-08-13 00:00:00,14831,518126.0,6d5760ccfd9748d3a1ab2ffaa72c938f,발로소득
8,2024-08-06 00:00:00,0,0.0,6d5760ccfd9748d3a1ab2ffaa72cTEST,[테스트] 발로소득
9,2024-08-07 00:00:00,0,0.0,6d5760ccfd9748d3a1ab2ffaa72cTEST,[테스트] 발로소득


In [207]:
start_date = "2024-05-23"
end_date = "2024-08-21"
platforms = {
    'ios':'d3a74845654442c38a77a1579c94e078',
    'aos':'6d5760ccfd9748d3a1ab2ffaa72c938f'
}
app_id = ''

In [206]:
#7일씩 가져옴. 마지막 날짜 기준
login_url = "https://partner.avatye.com/api/auth/check"
data_url = f"https://partner.avatye.com/api/cashButton/report/list?appID={app_id}&startDate={start_date}&endDate={end_date}&limit=999"
login_data = {
    'emailID' : "iehong@hecto.co.kr",
    'password' : "cash!1234"
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0',
    'Content-Type': 'application/json', 
}
def get_login(login_url,json,headers):
    response = session.post(login_url, json=login_data, headers=headers)
    if response.status_code == 200:
        return response
    else:
        False
        
# 웹 페이지 URL
def get_data(data_url):
    url = data_url
    # 웹 페이지의 HTML 가져오기
    response = requests.get(url)
    print(response)
    return response.json()

def convert_timezone(utc) :
    utc_time = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%S.%fZ")
    kst_time = utc_time + timedelta(hours=9)
    return kst_time.strftime("%Y-%m-%d %H:%M:%S")

def get_avati_report(json_data):
    df = pd.json_normalize(
        json_data['list'],
        # record_path=['data'],
        # meta=['appID', 'appName'],
        # errors='ignore'
    )

    df['actionDateTime'] = df['actionDateTime'].apply(convert_timezone)

    return df

with requests.Session() as session:
    response = get_login(login_url, json=login_data, headers=headers)
    if response == False:
        print("login_failed")
        sys.exit(1)
    else:
        print("login_success")
        pass
    
    json_data = get_data(data_url)
    print(json_data)

    result_data = get_avati_report(json_data)

result_data
    




login_success
<Response [200]>
{'list': [{'actionDateTime': '2024-08-19T15:00:00.000Z', 'newAccount': 107, 'dau': 1489, 'appDau': 40557, 'touchCount': 152128, 'touchPerUser': 102, 'invitedCount': None, 'totalRevenue': 30773, 'totalExpense': 17000, 'expectedRevenue': 30773, 'totalSales': 30773}, {'actionDateTime': '2024-08-18T15:00:00.000Z', 'newAccount': 145, 'dau': 1584, 'appDau': 40684, 'touchCount': 157295, 'touchPerUser': 99, 'invitedCount': None, 'totalRevenue': 39234, 'totalExpense': 24000, 'expectedRevenue': 39234, 'totalSales': 39234}, {'actionDateTime': '2024-08-17T15:00:00.000Z', 'newAccount': 149, 'dau': 1452, 'appDau': 37518, 'touchCount': 145297, 'touchPerUser': 100, 'invitedCount': None, 'totalRevenue': 40815, 'totalExpense': 17000, 'expectedRevenue': 40815, 'totalSales': 40815}, {'actionDateTime': '2024-08-16T15:00:00.000Z', 'newAccount': 103, 'dau': 1333, 'appDau': 37261, 'touchCount': 139061, 'touchPerUser': 104, 'invitedCount': None, 'totalRevenue': 41444, 'totalExpen

,actionDateTime,newAccount,dau,appDau,touchCount,touchPerUser,invitedCount,totalRevenue,totalExpense,expectedRevenue,totalSales
0,2024-08-20 00:00:00,107,1489,40557,152128,102.0,None,30773,17000,30773,30773
1,2024-08-19 00:00:00,145,1584,40684,157295,99.0,None,39234,24000,39234,39234
2,2024-08-18 00:00:00,149,1452,37518,145297,100.0,None,40815,17000,40815,40815
3,2024-08-17 00:00:00,103,1333,37261,139061,104.0,None,41444,16000,41444,41444
4,2024-08-16 00:00:00,104,1372,39198,143229,104.0,None,37644,15000,37644,37644
...,...,...,...,...,...,...,...,...,...,...,...
84,2024-05-28 00:00:00,0,0,0,0,NaN,None,0,0,0,0
85,2024-05-27 00:00:00,0,2,0,7,3.0,None,0,4000,0,0
86,2024-05-26 00:00:00,0,0,0,0,NaN,None,0,0,0,0
87,2024-05-25 00:00:00,0,3,0,8,2.0,None,0,1000,0,0


애드팝콘 revenue api

In [128]:
token = pd.read_csv('./access.csv').loc[0]['token']
start_date = date.today() - timedelta(days=120)
end_date = date.today() - timedelta(days=1)
start_date = start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

start_date = "2024-08-01"
end_date = "2024-08-31"


print(start_date, end_date)

def get_json_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # 요청이 성공하지 못하면 예외 발생
        data = response.json()  # 응답을 JSON 형식으로 파싱
        return data
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # HTTP 오류 처리
    except Exception as err:
        print(f"An error occurred: {err}")  # 기타 오류 처리
    return None

def pretty_print_json(data):
    if data:
        pretty_json = json.dumps(data, indent=4)
        print(pretty_json)

url = f"https://report.ad-brix.com/v1/Revenue?access_token={token}&startDate={start_date}&endDate={end_date}&format=json"
json_data = get_json_from_url(url)

df = pd.json_normalize(
    json_data['Datas']
)    

df['ReportDate'] = pd.to_datetime(df['ReportDate'])


temp_df = pd.concat([temp,df],axis=0)


2024-08-01 2024-08-31


In [133]:
temp_df['ReportDate'] = pd.to_datetime(temp_df['ReportDate'])

In [135]:
temp_df = temp_df.query("ReportDate < '2024-08-20'")

https://report.ad-brix.com/v1/Revenue?access_token=81289be620467f6efd4e2d522febc1cd&startDate=2024-08-01&endDate=2024-08-02&format=json

> GCP 테이블 생성

In [168]:
from google.cloud import bigquery
from google.cloud.bigquery import job
import pandas as pd
import os
from tqdm import tqdm
import pandas as pd
from google.oauth2 import service_account
PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

In [171]:
def set_table(df, table_id):
    project_id = 'ballosodeuk'
    dataset_id = 'external_mart'
    table_id = table_id
    table_full_id = f'{project_id}.{dataset_id}.{table_id}'
    
    df.columns = df.columns.str.replace('.', '_')

    try:
        # BigQuery 클라이언트 생성
        client = bigquery.Client(project=project_id)
        
        # 테이블에 대한 job 설정
        job_config = bigquery.LoadJobConfig(
            write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # 기존 테이블을 대체
        )
        
        # DataFrame을 BigQuery 테이블로 업로드
        job = client.load_table_from_dataframe(df, table_full_id, job_config=job_config)
        job.result()  # 작업이 완료될 때까지 대기
        
        print("Table is set successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [172]:
set_table(temp_df,'offerwall_adpopcorn_optincome')

Table is set successfully.
